# MLP Clustering Significance Stability over Multiple Trainings - Different #Clusters

### Before Running on Perceptron

Go to `src/train_nn.py` to line 46, and change the path `./models/` to `/scratch/<your username>/models`

Do not forget to undo this change!

DO NOT COMMIT THIS CHANGE TO GITHUB!!!

In [1]:
N_CLUSTERS = 4

In [2]:
# Parameters
N_CLUSTERS = 7


In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
import sys
sys.path.append('..')

import random
import subprocess
from multiprocessing import Pool

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from tqdm import tqdm, trange
from IPython import display

from src.visualization import draw_mlp_clustering_report, run_double_spectral_cluster, run_spectral_cluster
from src.utils import get_weights_paths, build_clustering_results, cohen_d_stats
from src.experiment_tagging import get_model_path, MODEL_TAG_LOOKUP
# from src.spectral_cluster_model import SHUFFLE_METHODS
SHUFFLE_METHODS = ['layer', 'layer_nonzero']

In [5]:
N_TRAINED_MODELS = 10

# we ar doing 320 experiments (=32 models x 10 times),
# and let's have expectation of false-positive <= 1
# so we can set n_shuffles to 320, and then p-values < 1/401
# if we make it too large, let see, 1000 shuffles...
# ... we might reduce the statistical power (given H1, the probability to reject it incorrectly)!
N_SHUFFLES = 320

N_GPUS = 4

TAGS_COMMANDS = {
    'MNIST': 'python -m src.train_nn with mlp_config dataset_name=mnist',
    'CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=cifar10 pruning_epochs=40',
#    'LINE': 'python -m src.train_nn with mlp_config dataset_name=line',
    'FASHION': 'python -m src.train_nn with mlp_config dataset_name=fashion',
    'MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist with_dropout=True',
    'CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=cifar10 epochs=100 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line with_dropout=True',
    'FASHION+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=fashion with_dropout=True',
#    'LINE-MNIST': 'python -m src.train_nn with mlp_config dataset_name=line-mnist',
#    'LINE-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40',
#    'LINE-MNIST-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated',
#    'LINE-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40',
#    'LINE-MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist with_dropout=True',
#    'LINE-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE-MNIST-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated with_dropout=True',
#    'LINE-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'RANDOM': 'python -m src.train_nn with mlp_config dataset_name=random',
#    'RANDOM+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random with_dropout=True',
#    'MNIST-x1.5-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30',
#    'MNIST-x1.5-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30 with_dropout=True',
#    'MNIST-x2-EPOCHS':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40',
#    'MNIST-x2-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40 with_dropout=True',
#    'MNIST-x10-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200',
#    'MNIST-x10-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200 with_dropout=True',
#    'RANDOM-x50-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000',
#    'RANDOM-x50-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000 with_dropout=True',
#    'RANDOM-OVERFITTING': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000',
#    'RANDOM-OVERFITTING+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000 with_dropout=True'
}
DATASETS_TAGS = [command.split()[5][13:] for command in TAGS_COMMANDS.values()]

In [6]:
if False:
    def train_model_multiply(command, n_models_per_command, gpu_id=None):    
        for _ in range(n_models_per_command):
            actual_command = f'cd .. && {command}'#' > /dev/null 2>&1'
            print(actual_command)
            subprocess.run(actual_command, shell=True,
                           env={'CUDA_VISIBLE_DEVICES': str(gpu_id)})


    def train_models(commands, n_models_per_command, gpu_id=None):
        for command in commands:
            train_model_multiply(command, n_models_per_command, gpu_id)

            commands = list(TAGS_COMMANDS.values())
    random.shuffle(commands)

    assert len(commands) % N_GPUS == 0
    n_commands_per_gpu = len(commands) // N_GPUS

    with Pool(N_GPUS) as p:

        p.starmap(train_models,
                 [(commands[gpu_id*n_commands_per_gpu : (gpu_id+1)*n_commands_per_gpu], N_TRAINED_MODELS, gpu_id)
                 for gpu_id in range(N_GPUS)])

In [7]:
results = {}

for shuffle_method in SHUFFLE_METHODS:
    results[shuffle_method] = {}
    
    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')


        results[shuffle_method][model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-pruned-weights.pckl',
                                               n_samples=N_SHUFFLES,
                                               shuffle_method=shuffle_method,
                                               n_clusters=N_CLUSTERS)
            for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [03:36<32:32, 216.95s/it]

 20%|██        | 2/10 [07:12<28:51, 216.47s/it]

 30%|███       | 3/10 [10:34<24:45, 212.20s/it]

...

 70%|███████   | 7/10 [23:09<10:01, 200.39s/it]

 80%|████████  | 8/10 [26:23<06:36, 198.42s/it]

 90%|█████████ | 9/10 [29:57<03:23, 203.17s/it]

100%|██████████| 10/10 [33:21<00:00, 203.40s/it]

100%|██████████| 10/10 [33:21<00:00, 200.15s/it]

In [8]:
run_spectral_cluster?

In [9]:
def build_all_models_per_shuffle_method_df(single_shuffle_method_results):
    model_dfs = []

    for model_tag, model_results in single_shuffle_method_results.items():
        _, metrics  = zip(*model_results)
        model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    df = pd.concat(model_dfs, ignore_index=True)

    df['is_sig'] = np.isclose(df['percentile'], 1 / (N_SHUFFLES + 1))
    
    return df

layer_df = build_all_models_per_shuffle_method_df(results['layer'])
layer_nonzero_df = build_all_models_per_shuffle_method_df(results['layer_nonzero'])

In [10]:
layer_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,0.255226,4.701298,320,4.634377,0.003115,0.015993,0.9835,0.095332,1.0,0.000048,-4.184349,MNIST,True
1,0.269979,4.707760,320,4.545577,0.003115,0.024822,0.9847,0.090697,1.0,0.000056,-6.533918,MNIST,True
2,0.282360,4.704236,320,4.473643,0.003115,0.026848,0.9830,0.099833,1.0,0.000059,-8.588974,MNIST,True
3,0.267632,4.701830,320,4.559476,0.003115,0.024402,0.9828,0.099546,1.0,0.000062,-5.833708,MNIST,True
4,0.259061,4.724072,320,4.610962,0.003115,0.019739,0.9845,0.097756,1.0,0.000081,-5.730319,MNIST,True


In [11]:
layer_nonzero_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,0.255226,4.649677,320,4.634377,0.376947,0.047088,0.9835,0.095332,1.0,0.000048,-0.324925,MNIST,False
1,0.269979,4.693166,320,4.545577,0.003115,0.058114,0.9847,0.090697,1.0,0.000056,-2.539635,MNIST,True
2,0.282360,4.560108,320,4.473643,0.003115,0.024097,0.9830,0.099833,1.0,0.000059,-3.588255,MNIST,True
3,0.267632,4.619620,320,4.559476,0.065421,0.038410,0.9828,0.099546,1.0,0.000062,-1.565863,MNIST,False
4,0.259061,4.675526,320,4.610962,0.283489,0.080500,0.9845,0.097756,1.0,0.000081,-0.802033,MNIST,False


In [12]:
MODEL_DATA_COLUMNS = ['model',
                      'train_acc', 'train_loss', 'test_acc', 'test_loss',
                      'ncut', 'ave_in_out',
                      'n_samples']

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

SHUFFLING_DATA_COLUMNS = ['mean', 'percentile', 'stdev', 'z_score', 'is_sig']

two_shuffling_method_df = pd.merge(layer_df[SHUFFLING_DATA_COLUMNS],
                                     layer_nonzero_df[SHUFFLING_DATA_COLUMNS],
                                     left_index=True, right_index=True,
                                     suffixes=('_layer', '_layer_nonzero'))

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

df = pd.merge(layer_df[MODEL_DATA_COLUMNS], two_shuffling_method_df,
              left_index=True, right_index=True,)

df['cohen_d'] = df.apply(lambda r: cohen_d_stats(r['mean_layer'], r['stdev_layer'], r['n_samples'],
                                 r['mean_layer_nonzero'], r['stdev_layer_nonzero'], r['n_samples']),
                                axis=1)

df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,4.634377,0.255226,320,4.701298,0.003115,0.015993,-4.184349,True,4.649677,0.376947,0.047088,-0.324925,False,0.771166
1,MNIST,1.000000,0.000056,0.9847,0.090697,4.545577,0.269979,320,4.707760,0.003115,0.024822,-6.533918,True,4.693166,0.003115,0.058114,-2.539635,True,0.732510
2,MNIST,1.000000,0.000059,0.9830,0.099833,4.473643,0.282360,320,4.704236,0.003115,0.026848,-8.588974,True,4.560108,0.003115,0.024097,-3.588255,True,0.873361
3,MNIST,1.000000,0.000062,0.9828,0.099546,4.559476,0.267632,320,4.701830,0.003115,0.024402,-5.833708,True,4.619620,0.065421,0.038410,-1.565863,False,0.804074
4,MNIST,1.000000,0.000081,0.9845,0.097756,4.610962,0.259061,320,4.724072,0.003115,0.019739,-5.730319,True,4.675526,0.283489,0.080500,-0.802033,False,0.679664
5,MNIST,1.000000,0.000048,0.9835,0.095626,4.498175,0.278093,320,4.726294,0.003115,0.021361,-10.679443,True,4.649077,0.003115,0.038872,-3.881992,True,0.803786
6,MNIST,1.000000,0.000056,0.9838,0.087238,4.514373,0.275301,320,4.695680,0.003115,0.024391,-7.433439,True,4.704034,0.003115,0.145137,-1.306768,True,0.562472
7,MNIST,1.000000,0.000061,0.9838,0.100444,4.592311,0.262143,320,4.716121,0.003115,0.022920,-5.401807,True,4.707152,0.003115,0.055481,-2.069906,True,0.740625
8,MNIST,1.000000,0.000047,0.9828,0.093142,4.536716,0.271483,320,4.699434,0.003115,0.028623,-5.684835,True,4.573713,0.065421,0.026173,-1.413515,False,0.860314
9,MNIST,1.000000,0.000049,0.9833,0.102283,4.589832,0.262555,320,4.724979,0.003115,0.026312,-5.136238,True,4.700492,0.003115,0.035032,-3.158842,True,0.811677


In [13]:
# df = pd.read_csv('../results/mlp-clustering-stability-two-shuffling-methods-all-samples.csv' index_col=0)

In [14]:
df_grpby_model = df.groupby('model')

TWO_SHUFFLE_STATS_COLUMNS = [f'{stat}_{shuffle_method}'
                             for stat in ['mean', 'stdev', 'z_score', 'percentile', 'is_sig']
                             for shuffle_method in SHUFFLE_METHODS]

statistics_df = pd.concat([(df_grpby_model[['ncut'] + TWO_SHUFFLE_STATS_COLUMNS]
                             .agg(['mean', 'std'])),
                            df_grpby_model.size().rename('n_models'),
                            (df_grpby_model[['train_acc', 'test_acc']]
                             .agg('mean'))],
                           axis=1)

In [15]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(percentile_layer, std)","(percentile_layer_nonzero, mean)","(percentile_layer_nonzero, std)","(is_sig_layer, mean)","(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc
model,,,,,,,,,,,,,,,,,,,,,
CIFAR10,4.707068,0.101122,4.647285,0.008580,4.632797,0.095489,0.022622,0.002052,0.095566,0.093486,...,0.363062,0.716511,0.236090,0.1,0.316228,0.0,0.000000,10,0.650088,0.41523
CIFAR10+DROPOUT,4.372795,0.159841,4.618436,0.031946,4.465084,0.135004,0.018004,0.004412,0.111580,0.027130,...,0.315243,0.242991,0.234532,0.9,0.316228,0.2,0.421637,10,0.427050,0.42189
FASHION,4.351071,0.057357,4.613037,0.013606,4.457097,0.054333,0.025383,0.003354,0.101812,0.040492,...,0.000000,0.140187,0.250171,1.0,0.000000,0.0,0.000000,10,0.982630,0.89344
FASHION+DROPOUT,4.079355,0.046460,4.662912,0.009527,4.239097,0.038359,0.027192,0.003525,0.076729,0.053602,...,0.000000,0.021807,0.021782,1.0,0.000000,0.5,0.527046,10,0.863268,0.86921
MNIST,4.555544,0.051615,4.710170,0.011655,4.653256,0.053716,0.023541,0.003714,0.054891,0.035851,...,0.000000,0.080997,0.135593,1.0,0.000000,0.6,0.516398,10,1.000000,0.98357
MNIST+DROPOUT,4.222384,0.035363,4.711981,0.006176,4.343858,0.025145,0.022634,0.002080,0.027524,0.027864,...,0.000000,0.003115,0.000000,1.0,0.000000,1.0,0.000000,10,0.966788,0.97940


In [16]:
# TODO: ask for a code review
# std = np.sqrt(np.mean(x**2) - np.mean(x)**2)

def calc_overall_stdev(row, suffix=''):
    return np.sqrt(                 # 5. apply root square to get the SD from the variance
        (np.mean(                   # 3. divide by the number of trained model
            (row[f'stdev{suffix}']**2)       # 1. cancel the overall square root
             + row[f'mean{suffix}']**2))     # 2. add the mean and get the sum or squares
        - np.mean(row[f'mean{suffix}'])**2)  # 4. substrct the square of the overall mean

In [17]:
SHUFFLE_METHOD_SUFFIX = ['_layer', '_layer_nonzero']
for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'overall_std{suffix}'] = df_grpby_model.apply(calc_overall_stdev, suffix)

for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'is_stable{suffix}'] = (statistics_df[(f'is_sig{suffix}', 'mean')] == 1)

statistics_df = statistics_df.loc[TAGS_COMMANDS.keys(), :]

In [18]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,4.555544,0.051615,4.710170,0.011655,4.653256,0.053716,0.023541,0.003714,0.054891,0.035851,...,0.000000,0.6,0.516398,10,1.000000,0.98357,0.026246,0.082259,True,False
CIFAR10,4.707068,0.101122,4.647285,0.008580,4.632797,0.095489,0.022622,0.002052,0.095566,0.093486,...,0.316228,0.0,0.000000,10,0.650088,0.41523,0.024120,0.158760,False,False
FASHION,4.351071,0.057357,4.613037,0.013606,4.457097,0.054333,0.025383,0.003354,0.101812,0.040492,...,0.000000,0.0,0.000000,10,0.982630,0.89344,0.028654,0.120409,True,False
MNIST+DROPOUT,4.222384,0.035363,4.711981,0.006176,4.343858,0.025145,0.022634,0.002080,0.027524,0.027864,...,0.000000,1.0,0.000000,10,0.966788,0.97940,0.023463,0.045004,True,True
CIFAR10+DROPOUT,4.372795,0.159841,4.618436,0.031946,4.465084,0.135004,0.018004,0.004412,0.111580,0.027130,...,0.316228,0.2,0.421637,10,0.427050,0.42189,0.035499,0.171802,False,False
FASHION+DROPOUT,4.079355,0.046460,4.662912,0.009527,4.239097,0.038359,0.027192,0.003525,0.076729,0.053602,...,0.000000,0.5,0.527046,10,0.863268,0.86921,0.028849,0.098982,True,False


In [19]:
df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-all-samples-K-{N_CLUSTERS}.csv')
statistics_df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-statistics-K-{N_CLUSTERS}.csv')

### Model Stability Statistics

In [20]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,4.555544,0.051615,4.710170,0.011655,4.653256,0.053716,0.023541,0.003714,0.054891,0.035851,...,0.000000,0.6,0.516398,10,1.000000,0.98357,0.026246,0.082259,True,False
CIFAR10,4.707068,0.101122,4.647285,0.008580,4.632797,0.095489,0.022622,0.002052,0.095566,0.093486,...,0.316228,0.0,0.000000,10,0.650088,0.41523,0.024120,0.158760,False,False
FASHION,4.351071,0.057357,4.613037,0.013606,4.457097,0.054333,0.025383,0.003354,0.101812,0.040492,...,0.000000,0.0,0.000000,10,0.982630,0.89344,0.028654,0.120409,True,False
MNIST+DROPOUT,4.222384,0.035363,4.711981,0.006176,4.343858,0.025145,0.022634,0.002080,0.027524,0.027864,...,0.000000,1.0,0.000000,10,0.966788,0.97940,0.023463,0.045004,True,True
CIFAR10+DROPOUT,4.372795,0.159841,4.618436,0.031946,4.465084,0.135004,0.018004,0.004412,0.111580,0.027130,...,0.316228,0.2,0.421637,10,0.427050,0.42189,0.035499,0.171802,False,False
FASHION+DROPOUT,4.079355,0.046460,4.662912,0.009527,4.239097,0.038359,0.027192,0.003525,0.076729,0.053602,...,0.000000,0.5,0.527046,10,0.863268,0.86921,0.028849,0.098982,True,False


### Appendix - All Samples

In [21]:
df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,4.634377,0.255226,320,4.701298,0.003115,0.015993,-4.184349,True,4.649677,0.376947,0.047088,-0.324925,False,0.771166
1,MNIST,1.000000,0.000056,0.9847,0.090697,4.545577,0.269979,320,4.707760,0.003115,0.024822,-6.533918,True,4.693166,0.003115,0.058114,-2.539635,True,0.732510
2,MNIST,1.000000,0.000059,0.9830,0.099833,4.473643,0.282360,320,4.704236,0.003115,0.026848,-8.588974,True,4.560108,0.003115,0.024097,-3.588255,True,0.873361
3,MNIST,1.000000,0.000062,0.9828,0.099546,4.559476,0.267632,320,4.701830,0.003115,0.024402,-5.833708,True,4.619620,0.065421,0.038410,-1.565863,False,0.804074
4,MNIST,1.000000,0.000081,0.9845,0.097756,4.610962,0.259061,320,4.724072,0.003115,0.019739,-5.730319,True,4.675526,0.283489,0.080500,-0.802033,False,0.679664
5,MNIST,1.000000,0.000048,0.9835,0.095626,4.498175,0.278093,320,4.726294,0.003115,0.021361,-10.679443,True,4.649077,0.003115,0.038872,-3.881992,True,0.803786
6,MNIST,1.000000,0.000056,0.9838,0.087238,4.514373,0.275301,320,4.695680,0.003115,0.024391,-7.433439,True,4.704034,0.003115,0.145137,-1.306768,True,0.562472
7,MNIST,1.000000,0.000061,0.9838,0.100444,4.592311,0.262143,320,4.716121,0.003115,0.022920,-5.401807,True,4.707152,0.003115,0.055481,-2.069906,True,0.740625
8,MNIST,1.000000,0.000047,0.9828,0.093142,4.536716,0.271483,320,4.699434,0.003115,0.028623,-5.684835,True,4.573713,0.065421,0.026173,-1.413515,False,0.860314
9,MNIST,1.000000,0.000049,0.9833,0.102283,4.589832,0.262555,320,4.724979,0.003115,0.026312,-5.136238,True,4.700492,0.003115,0.035032,-3.158842,True,0.811677


### Unpruned results without shuffling
#### TODO: Refactor
- [ ] Combine with the unpruned table

In [22]:
if False:
    unpruned_results = {}

    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')
        unpruned_results[model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-unpruned-weights.pckl',
                                                   with_shuffle=False)
                for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

    unpruned_model_dfs = []

    for model_tag, model_results in unpruned_results.items():
        _, metrics  = zip(*model_results)
        unpruned_model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    unpruned_df = pd.concat(unpruned_model_dfs, ignore_index=True)

    unpruned_df_grpby_model = unpruned_df.groupby('model')


    unpruned_statistics_df = pd.concat([(unpruned_df_grpby_model[['ncut']]
                                 .agg(['mean', 'std'])),
                                unpruned_df_grpby_model.size().rename('n_models'),
                                (unpruned_df_grpby_model[['train_acc', 'test_acc']]
                                 .agg('mean'))],
                               axis=1)

    unpruned_statistics_df = unpruned_statistics_df.loc[TAGS_COMMANDS.keys(), :]

    unpruned_statistics_df

In [23]:
# unpruned_df.to_csv(f'../results/mlp-clustering-stability-all-samples-unpruned-K-{N_CLUSTERS}.csv')
# unpruned_statistics_df.to_csv(f'../results/mlp-clustering-stability-statistics-unpruned-K-{N_CLUSTERS}.csv')